<a href="https://colab.research.google.com/github/tysonjohn015/P03_AI_Multivariate_Data_Analysis/blob/main/P3_01_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib
import zipfile
import re
import itertools

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import seaborn as sns

import plotly.offline as pyo
import plotly.express as px

import scipy.stats as st
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import display, Markdown

import missingno as msno
from wordcloud import WordCloud

from nltk.corpus import stopwords
from IPython.display import HTML

In [ ]:
# Initialisation des options d'affichage
pd.set_option('display.max_columns', None)
sns.set(font_scale=1.2)
pyo.init_notebook_mode()

In [ ]:
def print_md(text):
    """Affichage d'un text en Markdown"""
    display(Markdown(text))

def print_md_df_shape(df, df_origin=None):
    """
    Affichage des dimension de l'échantillon.
    Comparison avec l'échnatillon d'origine.
    """
    
    text = "Dimensions de l'échantillon :\n"
    
    if df_origin is not None:
        origin_rows_pct = int(df.shape[0] * 100 / df_origin.shape[0])
        origin_cols_pct = int(df.shape[1] * 100 / df_origin.shape[1])
        
        text += f"- **Nombre d'individus** : {df.shape[0]} ({origin_rows_pct}% du dataset complet)\n"
        text += f"- **Nombre de variables** : {df.shape[1]} ({origin_cols_pct}% du dataset complet)\n"
    else:
        text += f"- **Nombre d'individus** : {df.shape[0]}\n"
        text += f"- **Nombre de variables** : {df.shape[1]}\n"

    print_md(text)

def get_cat_var_emp_dist_df(df, c, k=None):
    """Renvoie la distribution empirique d'une variable de type catégorie"""
    if k is None:
        effectifs = list(df[c].value_counts().values)
        labels = df[c].value_counts().index.to_list()
    else:
        effectifs = list(df[c].value_counts().iloc[:k].values)
        labels = df[c].value_counts().iloc[:k].index.to_list()
        
        # Si il reste des modalités, on les aggrège ensemble
        effectif = df[c].value_counts().iloc[k:].sum()
        if effectif > 0:
            effectifs.append(effectif)
            labels.append("autres")

    df_tmp = pd.DataFrame({
        c: labels,
        "effectif": effectifs
    })
    
    df_tmp["f"] = df_tmp["effectif"] / df[c].count()
    df_tmp["F"] = df_tmp["f"].cumsum()
    
    return df_tmp

def categ_plot_pie_chart(df, c, k):
    """Affiche un pie chart de la distribution d'une variable de type catégorie"""
    df_tmp = get_cat_var_emp_dist_df(df, c, k)

    # On crée le pie chart
    fig = px.pie(
        df_tmp,
        values="effectif",
        names=c,
        title=f"Distribution de la variable {c}"
    )
    fig.update_traces(textinfo='percent+label')
    # fig.show()
    return HTML(fig.to_html())

def categ_plot_bar_chart(df, c, k, log=False):
    """Affiche un bar chart de la distribution d'une variable de type catégorie"""
    df_tmp = get_cat_var_emp_dist_df(df, c, k)

    # On crée le bar chart
    fig = px.bar(
        df_tmp,
        y='effectif',
        x=c,
        text='effectif',
        title=f"Distribution de la variable {c}",
        log_y=log
    )
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    # fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    # fig.show()
    return HTML(fig.to_html())

def num_categ_box_chart(
        df_clean,
        num_col,
        cat_col,
        title="",
        xlabel="",
        ylabel="",
        showfliers=True,
        rotation=0
    ):
    """
    Affiche un box chart d'une variable numérique en fonction des modalités
    d"une variable de type catégorie.
    """
    fig, ax = plt.subplots(figsize=(16, 16))

    meanprops = {
        'marker':'o',
        'markeredgecolor':'black',
        'markerfacecolor':'firebrick'
    }

    sns.boxplot(
        data=df_clean.sort_values(cat_col),
        x=cat_col,
        y=num_col,
        showfliers=showfliers,
        showmeans=True,
        meanprops=meanprops,
        ax=ax
    )

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    if rotation > 0:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=rotation)

    plt.show()

# Présentation générale du jeu de données de Open Food Facts

L'agence Santé publique France souhaite rendre les données de santé publique plus accessibles, pour qu’elles soient utilisables par ses agents.

Pour cela, nous allons anlyser un jeu de données fourni gratuitement par [Open Food Facts](https://world.openfoodfacts.org/who-we-are).
Il s'agit d'un projet collaboratif qui vise à collecter des données sur des produits alimentaires que l'on retrouve en grande surface. Le projet a été initié en France mais il est depuis devenu international.

Le projet possède une page wiki avec des nombreuses informations sur son jeu de données : [wiki](https://wiki.openfoodfacts.org/Main_Page).

## Téléchargement du jeu de données

In [ ]:
# # Run this cell and select the kaggle.json file downloaded
# # from the Kaggle account settings page.
# from google.colab import files
# files.upload()

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

**Téléchargement du jeu de données**

In [ ]:
# Copy the stackoverflow data set locally.
!kaggle datasets download -d tysonjohn/openfoodfacts

In [ ]:
!unzip openfoodfacts.zip

In [ ]:
# Chargement des données dans un DataFrame
df = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t')

print_md_df_shape(df)

## Premières lignes

Observons les premières lignes de notre jeu de données :

In [ ]:
df.head(2)

## Valeurs manquantes

En observant ces premières lignes, on remarque que beaucoup de colonnes sont remplies de valeurs manquantes (NaN).

In [ ]:
# On calcul le pourcentage de valeurs manquantes par variable
df_na = df.isna().sum() * 100 / df.shape[0]

fig, ax = plt.subplots(figsize=(16, 12))

# On affiche l'histogramme des valeurs manquantes
hist = df_na.hist(bins=20, ax=ax)

ax.set_title("Distribution des valeurs manquantes des variables")
ax.set_xlabel("Pourcentage de valeurs manquantes")
ax.set_ylabel("Nombre de variables")

from matplotlib.colors import LinearSegmentedColormap

# On crée un code couleur qui fait preuve d'affordance
colors = ["mediumseagreen", "gold", "lightcoral"]
nodes = [0.0, 0.5, 1.0]
mycmap = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))

for i, p in enumerate(hist.patches):
  plt.setp(p, 'facecolor', mycmap(i/20))

# On ajoute la valeur de la première classe
h = hist.get_children()[0].get_height()
ax.text(1.2, h + 0.6, f"{int(h)}")

# On ajoute la valeur de la dernière classe
h = hist.get_children()[19].get_height()
ax.text(96, h + 0.6, f"{int(h)}")

xticks = list(range(0, 105, 5))
ax.set_xticks(xticks)
ax.set_xticklabels([str(x) for x in xticks])

# plt.savefig('distribution valeurs manquantes.png', bbox_inches='tight')
plt.show()

# Calcul du nombre total de valeurs manquates
na_nb = df.isna().sum().sum()

text = f"- **Valeurs manquantes** : {na_nb} sur {df.size} (soit {na_nb * 100 / df.size:0.2f}%)\n"

print_md(text)

In [ ]:
# ToDeleteCell: to analysing the above code
for i, p in enumerate(hist.patches):
  print(i, '**', p,'***', mycmap(i/20))

**On** a 108 variables (58% des variables) qui ont plus de 95% de valeurs manquantes. Seules 16 variables (8,6% des variables) ont moins de 5% de valeurs manquantes.

Pour rappel, ce jeu de données est libre d'accès et peut être rempli par n'importe qui. De plus, certaines variables sont rarement disponibles sur les emballages des produits comme par exemple l'indice glycémique.

## Colonnes doublons

En observant chaques variables, on remarque que certaines ont le même nom racine comme :
- `countries`
- `countries_tags`
- `countries_fr`

Ou encore :
- `last_modified_t`:
- `last_modified_datetime`

Ces variables représentent la même information mais sous un format différent ([en savoir plus](https://static.openfoodfacts.org/data/data-fields.txt)).

## Suppression de variables inutiles

Commençons par supprimer les variables qui ont plus de 95% de valeurs manquantes :

In [ ]:
# Initializing
rows = []
df_clean = df

# On va analyser chaque colonne
for c in df_clean.columns:
    row = {}
    row["Valeurs manquantes"] = df_clean[c].isna().sum() * 100 / df_clean.shape[0]
    row["Valeurs nulles"] = ((df_clean[c] == 0) | (df_clean[c] == "")).sum() * 100 / df_clean.shape[0]
    row["Valeurs non nulles"] = 100 - (row["Valeurs manquantes"] + row["Valeurs nulles"])
    
    rows.append(row)

# On met le tout dans un DataFrame et on filtre par valeurs manquantes
df_columns = pd.DataFrame(rows, index=df_clean.columns)
df_columns = df_columns.sort_values("Valeurs manquantes", ascending=False)

In [ ]:
# ToDeleteCell : 
# df_columns[df_columns['Valeurs manquantes'] >= 95].shape
df_columns[df_columns['Valeurs manquantes'] >= 95].index.to_list()

In [ ]:
# On récupère la liste des variables à supprimer
cols_to_drop = df_columns[df_columns["Valeurs manquantes"] > 95].index.to_list()

# On supprime les variables
df_clean = df_clean.drop(columns=cols_to_drop)
df_columns = df_columns.drop(index=cols_to_drop)

print_md_df_shape(df_clean, df)

Regardons les variables ayant plus de 50% de valeurs manquantes :

In [ ]:
fig, ax = plt.subplots(figsize=(16,32))

# On affiche les indicateurs statistiques des variables ayant plus de 50% de valeurs manquantes
df_columns[df_columns["Valeurs manquantes"] > 50].plot.barh(
    stacked=True,
    ax=ax,
    fontsize=16,
    color={
        "Valeurs manquantes": "lightcoral",
        "Valeurs nulles": "gold",
        "Valeurs non nulles": "mediumseagreen",
    }
)

# On trace une ligne indiquant 75% de la population
ax.axvline(75, color="grey", ls="--", alpha=0.6, label="75% des individus")

ax.set_title("Représentation des valeurs manquantes", fontsize=16)
ax.set_xlabel("Pourcentage des individus")

plt.legend(loc='upper left', fontsize=16)

plt.show()

Nous allons supprimer les variables qui ont plus de 75% de valeurs manquantes car la grande majorité ne va pas nous intéresser pour l'analyse. Parmis ces variables, nous allons cependant conserver `additives_fr`,`allergens`.

Nous constatons aussi que certaines variables ont énromément de valeurs nulles comme `ingredients_from_palm_oil_n` ou encore `ingredients_that_may_be_from_palm_oil_n`. Nous allons supprimer ces variables par la suite.

In [ ]:
# ToDeleteCell
df_columns[df_columns["Valeurs manquantes"] <= 75.].index.to_list()

In [ ]:
# On récupère la liste des variables ayant plus de 75% de valeurs manquantes
cols_to_drop = df_columns[df_columns["Valeurs manquantes"] > 75.].index.to_list()

# Variables que l'on souhaite conserver même si elles ont beaucoup de valeurs manquantes
cols_to_keep = [
    "additives_fr",
    "allergens"
]

# Variables à supprimer
cols_to_drop = [c for c in cols_to_drop if c not in cols_to_keep]

# Suppression des variables
df_clean = df_clean.drop(columns=cols_to_drop)
df_columns = df_columns.drop(index=cols_to_drop)

print_md_df_shape(df_clean, df)

Regardons les variables représentant des catégories de produits :

In [ ]:
product_cat_cols = [
    "categories",
    "categories_tags",
    "categories_fr",
    "pnns_groups_1",
    "pnns_groups_2",
]

df_cat = df_clean[product_cat_cols].describe().T
df_cat["missing values"] = df_clean[product_cat_cols].isna().sum()

display(df_cat)

Supprimons les modalités `unknown` afin qu'elles ne soient plus prises en compte dans le calcul de nos indicateurs statistiques.

In [ ]:
# On commence par supprimer les modalités "unknown" pour qu'elles ne soient pas prises en compte
df_clean.loc[:, product_cat_cols] = df_clean[product_cat_cols].replace("unknown", np.nan)

df_cat = df_clean[product_cat_cols].describe().T
df_cat["missing values"] = df_clean[product_cat_cols].isna().sum().values

display(df_cat)

Les variables `categor.y.ies*` contiennent beaucoup trop de modalités.
Nous voudrions pourvoir classer nos produits en quelques dizaines de catégories au maximum.

Les variables `pnns_groups_1` et `pnns_groups_2` ont toutes les 2 un nombre raisonnable de modalités.

In [ ]:
def print_md_dist_emp_pnns_groups(df):
    outs = []

    for column in ["pnns_groups_1", "pnns_groups_2"]:
        outs.append(widgets.Output())

        with outs[-1]:
            df_distCat = get_cat_var_emp_dist_df(df, column)

            print_md(f"<center>Distribution empirique des {df_distCat.shape[0]} principales</br>modalités de {column}</center>")
            display(df_distCat)

    hbox = widgets.HBox(outs)
    hbox.layout.justify_content="space-around"
    
    display(hbox)

print_md_dist_emp_pnns_groups(df_clean)

On remarque qu'il y a des doublons dans les modalités. On peut facilement corriger cela en remplaçant le caractère `-` par un espace et en mettant les premières lettres des modalités en majuscule.

In [ ]:
for c in ["pnns_groups_1", "pnns_groups_2"]:
    df_clean[c] = df_clean[c].transform(lambda x: x.str.replace("-", " ").str.capitalize())

# Combine similar categories
df_clean["pnns_groups_2"] = df_clean["pnns_groups_2"].replace({
    "Pizza pies and quiche": "Pizza pies and quiches"
})

df_clean["pnns_groups_2"] = df_clean["pnns_groups_2"].replace({
    "Legumes": "Vegetables"
})

print_md_dist_emp_pnns_groups(df_clean)

`pnns_groups_1` contient 10 catégories de produits explicites et assez générales. `pnns_groups_2` en contient 38 qui sont plus détaillées. Nous allons conserver ces 2 variables.

Nous allons filtrer les variables doublons et les variables inutiles (trop de valeurs nulles etc...) :

In [ ]:
# On définit la liste des colonnes à conserver.
cols_to_keep = [
    "code",

    "creator",
    "last_modified_t",

    "product_name",
    "brands",
    "countries_fr",
    "pnns_groups_1",
    "pnns_groups_2",

    "nutriscore_grade",
    "nutriscore_score",

    "ingredients_text",
    "additives_n",
    "additives_fr",
    "allergens",

    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g"
]

# On filtre les variables que l'on souhaite conserver
df_clean = df_clean[cols_to_keep]

print_md_df_shape(df_clean, df)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

tmp = msno.bar(df_clean, ax=ax)
ax.set_title(
    """
    Représentation des valeurs présentes par variable.
    """,
    fontsize=16)
fig.axes[0].set_ylabel("Fréquence de valeurs présentes", fontsize=16)
fig.axes[1].set_ylabel("Nombre de valeurs présentes", fontsize=16)
plt.show()

## Suppression des doublons dans les individus

Commençons par observer les individus qui ont le même code bar :

In [ ]:
df_clean.shape

In [ ]:
# On récupère les doublons
doublons_df = df_clean[df_clean.duplicated("code", keep=False)]

# On affiche les premiers doublons
display(doublons_df.astype({"code": str}).sort_values("code").head(8))

Il s'agit effectivement de doublons.

Nous allons conserver les individus ayant la date de mise à jour la plus récente et nous allons compléter leurs valeurs manquantes avec les valeurs de leurs doublons.

In [ ]:
# On met à jour les valeurs manquantes des doublons
doublons_df = doublons_df.astype({"code": str}).sort_values(["code", "last_modified_t"]). \
    groupby("code", as_index=False). \
    fillna(method="ffill")

# On met à jour les doublons
df_clean.update(doublons_df)

# On supprime les doublons
df_clean = df_clean.sort_values("last_modified_t").drop_duplicates(subset="code")

print_md_df_shape(df_clean, df)

## Nettoyage des valeurs du tableau nutritionel

Obervons quelques indicateurs statistiques des variables nutritionelles :

In [ ]:
df_clean['energy_100g'][df_clean['energy_100g'].apply(lambda x: isinstance(x, str))]

In [ ]:
df_clean['energy_100g'] = pd.to_numeric(df_clean['energy_100g'])

In [ ]:
df_clean.describe().T

On remarque que certaines valeurs max sont trop grandes. Par exemple, la valeur max de l'énergie au 100g ne devrait pas dépasser les 3700kJ ([en savoir plus](https://fr.wikipedia.org/wiki/Valeur_%C3%A9nerg%C3%A9tique)).

Les valeurs aux 100g (excepté `energy_100g` qui est en kJ) ne devrait pas dépasser les 100g.

Il ne devrait aussi pas y avoir de valeurs négatives (sauf pour `nutriscore_score`).

Commençons par observer certains outliers de `energy_100g` :

In [ ]:
# Exemples de valeurs atypiques
display(df_clean[pd.to_numeric(df_clean["energy_100g"]) > 3700].head(2))

# Exemples de valeurs aberrantes
display(df_clean[pd.to_numeric(df_clean["energy_100g"]) > 4000].head(2))

Les valeurs affichées ci-dessus contiennent des valeurs atypique qui avoisinent les 3700kJ. Ce sont principalement des huiles qui contiennent 100% de matière grasse. Nous allons donc les conserver.

On observe aussi des valeurs beaucoup plus élevées qui sont certainement des valeurs aberrantes.

Nous allons clipper les valeurs inférieures à 3800kJ à 3700kJ. Les valeurs supérieures à 3800kJ seront considérés comme des valeurs aberrantes et seront supprimées.

In [ ]:
cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

all_cols = cols + ["energy_100g"]

# On supprime les valeurs aberrantes de l'énergie
df_clean.loc[df_clean["energy_100g"] < 0, all_cols] = np.nan
df_clean.loc[df_clean["energy_100g"] >= 3800, all_cols] = np.nan

# On clip les valeur atypiques à 3700kJ
df_clean["energy_100g"] = df_clean["energy_100g"].clip(upper=3700)

Voyons combien d'individus ont leur tableau nutritionnel complet :

In [ ]:
def print_md_notna_sizes():
    cols = [
        "energy_100g",
        "fat_100g",
        "saturated-fat_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fiber_100g",
        "proteins_100g",
        "salt_100g",
    ]

    text = "Voici le nombre d'individus dans les 2 sous-échantilllons suivants :\n"

    row_nb = df_clean.dropna(subset=cols).shape[0]
    row_pct = row_nb * 100 / df_clean.shape[0]
    text += f"- **Individus ayant leur tableau nutritionnel complet** : {row_nb} ({int(row_pct)}% des individus)\n"

    row_nb = df_clean.dropna(subset=cols + ["pnns_groups_2"]).shape[0]
    row_pct = row_nb * 100 / df_clean.shape[0]
    text += f"- **Individus ayant leur tableau nutritionnel complet et ayant la catégorie du produit** : {row_nb} ({int(row_pct)}% des individus)\n"

    print_md(text)

print_md_notna_sizes()

In [ ]:
def print_md_missing_val_nutri_table():
    cols = [
        "fat_100g",
        "carbohydrates_100g",
        "fiber_100g",
        "proteins_100g",
    ]

    text = "Nombre d'individus ayant leur valeur de l'énergie aux 100g "
    text += "et ayant des valeurs manquantes dans leur tableau nutritionnel :\n"

    tmp = df_clean[df_clean["energy_100g"].notna()][cols].isna().sum(axis=1)
    for i in range(4):
        tmp2 = (tmp == (i + 1)).sum()
        text += f"- Abscence de {i + 1} valeur(s) : {tmp2} individus\n"

    print_md(text)

print_md_missing_val_nutri_table()

Grâce à la formule de calcul de l'énergie au 100g, nous allons pouvoir compléter le tableau nutritionnel des individus ayant 1 seule valeur manquante :

In [ ]:
cols = [
    "fat_100g",
    "carbohydrates_100g",
    "fiber_100g",
    "proteins_100g",
]

# On crée un masque des individus ayant 1 seule valeur manquante
mask = df_clean["energy_100g"].notna() & (df_clean[cols].isna().sum(axis=1) == 1)

# Doubt : col_to_weight how its decided?
# On attribut à chaque variable son poids dans le calcul de l'énergie au 100g
col_to_weight = {
    "fat_100g": 37,
    "carbohydrates_100g": 17,
    "fiber_100g": 17,
    "proteins_100g": 8,
}

for c in col_to_weight:
    # On filtre les individus ayant la valeur de la variable c manquante
    mask2 = mask & df_clean[c].isna()
    
    # On enlève c de la formule
    ctw = col_to_weight.copy()
    # Doubt: what happening below ?
    del(ctw[c])
    # Doubt: what happening below ?
    df_clean.loc[mask2, c] = df_clean.loc[mask2, "energy_100g"]
    for c2, w in ctw.items():
        df_clean.loc[mask2, c] -= df_clean.loc[mask2, c2] * w

print_md_missing_val_nutri_table()

Nous allons clipper les valeurs des nutriments comprises entre 100 et 120g à 100g. Les valeurs supérieures à 120g ou négatives seront considérées comme des valeurs aberrantes et seront supprimées.

Nous allons aussi supprimer les valeurs des nutriments si leur somme est supérieure à 120g.

In [ ]:
df_clean[cols] > 120

In [ ]:
cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

all_cols = cols + ["energy_100g"]

# On supprime les valeurs aberrantes de l'énergie
df_clean.loc[df_clean["energy_100g"] < 0, all_cols] = np.nan
df_clean.loc[df_clean["energy_100g"] >= 3800, all_cols] = np.nan

# On clip les valeur atypiques à 3700kJ
df_clean["energy_100g"] = df_clean["energy_100g"].clip(upper=3700)

# On supprime les valeurs aberrantes des nutriments
df_clean.loc[(df_clean[cols] < 0).sum(axis=1) > 0, all_cols] = np.nan
df_clean.loc[(df_clean[cols] > 120).sum(axis=1) > 0, all_cols] = np.nan
    
# On clip les valeur atypiques à 100g
df_clean.loc[:, cols] = df_clean[cols].clip(lower=0, upper=100)

cols = [
    "fat_100g",
    "carbohydrates_100g",
    "fiber_100g",
    "proteins_100g",
]

# On supprime les valeurs aberrantes si la somme des nutriments > 120g
df_clean.loc[df_clean[cols].sum(axis=1) > 120, all_cols] = np.nan

Nous allons maintenant recalculer l'énergie au 100g grâce aux valeurs des nutriments.

Si la différence entre la valeur calculée et la valeur de `energy_100g` est inférieure à 5%, nous pourrons remplacer les valeurs vide du tableau nutritionnel de l'individu par 0.

In [ ]:
# Doubt : how did we come to this formula in the whole cell?
# On calcule l'énergie au 100g à partir des valeurs du tableau nutritionnel
df_clean["energy_100g_calc"] = df_clean["fat_100g"].fillna(0) * 37 + \
    df_clean["proteins_100g"].fillna(0) * 17 + \
    df_clean["carbohydrates_100g"].fillna(0) * 17 + \
    df_clean["fiber_100g"].fillna(0) * 8

# On calcule la distance normalisée entre les 2 énergies
df_clean["energy_100g_diff"] = (df_clean["energy_100g"] - df_clean["energy_100g_calc"]).abs()
df_clean["energy_100g_diff"] = df_clean["energy_100g_diff"] / (df_clean["energy_100g"] + 1e-6)

cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

# Si différence des énergie est inférieure à 5% alors on remplace les valeurs manquantes par 0
df_clean.loc[df_clean["energy_100g_diff"] < 0.05, cols].fillna(0, inplace=True)

# On supprime les colonnes temporaires
df_clean = df_clean.drop(columns=["energy_100g_calc", "energy_100g_diff"])

Enfin, nous allons supprimer les tableau nutritionnels dont toutes les valeurs sont à 0 :

In [ ]:
# Doubt : Why the df is filter for energy valu >= 8 ?
all_cols = [
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

cols = [
    "fat_100g",
    "carbohydrates_100g",
    "proteins_100g",
    "salt_100g",
]

# On sélectionne les produits ayant une énergie non nulle 
# afin de ne pas prendre en compte les eaux minérales par exemple.
mask = df_clean["energy_100g"] >= 8
# Parmis ces produits, on sélectionne ceux qui ont toutes leurs valeurs à 0
mask &= (df_clean[cols] == 0).all(axis=1)

# On supprime ces valeurs aberrantes
df_clean.loc[mask, all_cols] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

tmp = msno.bar(df_clean, ax=ax)
ax.set_title(
    """
    Représentation des valeurs présentes par variable.
    """,
    fontsize=16)
fig.axes[0].set_ylabel("Fréquence de valeurs présentes", fontsize=16)
fig.axes[1].set_ylabel("Nombre de valeurs présentes", fontsize=16)
plt.show()

In [ ]:
print_md_notna_sizes()

On constate que l'on a beaucoup plus de valeurs manquantes dans les valeurs nutritionnelles. Cela vient du fait que l'on a supprimé beaucoup de valeurs aberrantes. Ceci a été réalisé en utilisant des données métier. En effet, la répartition des nutriments peut être très variable d'un produit à l'autre même si ils ont la même appellation. On a donc préféré supprimer toutes données suspectes afin de pouvoir justement mettre en avant les différences entre les produits dans cette analyse.

On remarque cependant que l'on a plus d'individus ayant leur tableau nutritionnel complet ! On a pu reconstituer le tableau nutrionnel de **17%** des individus.

## Feature engineering

Les variables `product_name`, `ingredients_text`, `additives_fr` et `allergens` représentent respectivement les noms des produits, les listes d'ingrédients, les additifs et les produits allergènes. Ces données sont des listes de textes et ne respectent pas de format particulier. Le wiki du projet Open Food Facts explique qu'il est effectivement difficile de nettoyer ces données ([en savoir plus](https://wiki.openfoodfacts.org/Ingredients_Extraction_and_Analysis)).

Nous allons, à partir de ces variables, créer les variables suivantes qui seront plus facilement exploitables :
- `ingredients_n` : nombre d'ingrédients.
- `allergens_n` : nombre d'allergènes.
- `additifs_n_risque_faible` : nombre d'additifs ayant un risque de sur-exposition très faible ou nul.
- `additifs_n_risque_moyen` : nombre d'additifs ayant un risque modéré de sur-exposition.
- `additifs_n_risque_eleve` : nombre d'additifs ayant un risque élevé de sur-exposition.
- `pn_beverages` : lien entre le nom du produit et la catégorie des boissons.
- `pn_cereals_and_potatoes` : lien entre le nom du produit et la catégorie des céréales et pommes de terre.
- `pn_composite_foods` : lien entre le nom du produit et la catégorie des produits composés.
- `pn_fat_and_sauces` : lien entre le nom du produit et la catégorie des sauces et matières grasses.
- `pn_fish_meat_eggs` : lien entre le nom du produit et la catégorie des poissons, des viandes et des oeufs.
- `pn_fruits_and_vegetables` : lien entre le nom du produit et la catégorie des fruits et légumes.
- `pn_milk_and_dairy_products` : lien entre le nom du produit et la catégorie des produits laitiers.
- `pn_salty_snacks` : lien entre le nom du produit et la catégorie des snacks salés.
- `pn_sugary_snacks` : lien entre le nom du produit et la catégorie des snacks sucrés.

### Création de la variable `ingredients_n`

In [ ]:
# On convertit le texte représentant les ingrédients en une liste d'ingrédients
df_clean["ingredient"] = df_clean["ingredients_text"].transform(lambda x: x.str.split(","))

# On crée la nouvelle variable
df_clean["ingredients_n"] = df_clean["ingredient"].apply(lambda x: len(x) if isinstance(x, list) else x)

# Doubt: why transform done below ?
# On explose cette liste en créant un nouveau DataFrame avec une ligne par ingrédient
tmp = df_clean[["code", "ingredient"]].explode("ingredient").transform(lambda x: x.str.strip()).dropna()

# On supprime la variable créée temporairement
df_clean = df_clean.drop(columns=["ingredient"])

# On nettoie un peu la liste (suppression du padding, texte en minuscule)
tmp["ingredient"] = tmp["ingredient"].transform(lambda x: x.str.strip().str.lower())

# On sauvegarde le DataFrame qui pourra nous servir pour la suite de l'analyse
tmp.to_csv("df_clean_ingredients.csv")

In [ ]:
print_md("Voici des exemples de la nouvelle variable :")

df_clean[["ingredients_text", "ingredients_n"]].head()

### Création de la variable `allergens_n`

In [ ]:
# On convertit le texte représentant les allergènes en une liste d'allergènes
df_clean["allergen"] = df_clean["allergens"].transform(lambda x: x.str.split(","))

# On crée la nouvelle variable
df_clean["allergens_n"] = df_clean["allergen"].apply(lambda x: len(x) if isinstance(x, list) else x)

# La variable "allergens" ne contient pas de modalités pour indiquer qu'il n'y a pas
# d'allergène. Nous allons donc ajouter la valeur 0 pour les individus qui ont leur
# liste d'ingrédient et une valeur manquante pour "allergens"
mask = df_clean["ingredients_text"].notna() & df_clean["allergens"].isna()
df_clean.loc[mask, "allergens_n"] = 0

# On explose cette liste en créant un nouveau DataFrame avec une ligne par allergène
tmp = df_clean[["code", "allergen"]].explode("allergen").transform(lambda x: x.str.strip()).dropna()

# On supprime la variable créée temporairement
df_clean = df_clean.drop(columns=["allergen"])

# On nettoie un peu la liste (suppression du padding, texte en minuscule)
tmp["allergen"] = tmp["allergen"].transform(lambda x: x.str.strip().str.lower())

# On sauvegarde le DataFrame qui pourra nous servir pour la suite de l'analyse
tmp.to_csv("df_clean_allergens.csv")

In [ ]:
print_md("Voici des exemples de la nouvelle variable :")

df_clean.dropna(subset=["allergens"])[["allergens", "allergens_n"]].head()

### Création des variables `additifs_risque_*`

Pour créer ces variables, nous allons utiliser la liste des additifs disponibles sur Open Food Facts ([en savoir plus](https://world.openfoodfacts.org/additives)).

In [ ]:
additifs = pd.read_csv("additives.csv", sep=",")
additifs.head(2)

In [ ]:
# Commençons par supprimer les variables inutiles
additifs = additifs.drop(columns=["Code","Products"])

# On ajoute une ligne avec le code de l'additif
additifs["Additive_code"] = additifs["Additive"].apply(lambda x: x.split(" - ")[0])
additifs["Additive_code"] = additifs["Additive_code"].transform(lambda x: x.str.strip().str.lower())

additifs.head(2)

In [ ]:
# On convertit le texte représentant les additifs en une liste d'additifs
df_clean["additif"] = df_clean["additives_fr"].transform(lambda x: x.str.split(","))

# On explose cette liste en créant un nouveau DataFrame avec une ligne par additif
tmp = df_clean[["code", "additif"]].explode("additif").transform(lambda x: x.str.strip()).dropna()

# On supprime la variable créée temporairement
df_clean = df_clean.drop(columns=["additif"])

# Doubt : it was not clear how the below cleaning was spotted ?
# On nettoie un peu la liste (suppression du padding, suppression de l'entête "en:")
tmp["additif"] = tmp["additif"].transform(lambda x: x.str.strip().str.replace("^en:", ""))

# On ajoute une ligne avec le code de l'additif
tmp["additif_code"] = tmp["additif"].apply(lambda x: x.split(" - ")[0])
tmp["additif_code"] = tmp["additif_code"].transform(lambda x: x.str.strip().str.lower())

# On merge les 2 DataFrames
tmp = tmp.reset_index().merge(
    additifs[["Additive_code", "Risk"]],
    how="left",
    left_on="additif_code",
    right_on="Additive_code"
).set_index("index")

# On remplace les valeurs des riques par quelque chose de plus lisible
tmp["risque"] = tmp["Risk"].replace({
    "No or very low risk of over exposure": "faible",
    "Moderate risk of over exposure": "moyen",
    "High risk of over exposure": "eleve",
})

tmp = tmp[[
    "code",
    "additif",
    "additif_code",
    "risque"
]]

# On sauvegarde le DataFrame qui pourra nous servir pour la suite de l'analyse
tmp.to_csv("df_clean_additifs.csv")

tmp.head(2)

In [ ]:
# On convertit le risque en nombre
tmp["additifs_n_risque_faible"] = (tmp["risque"] == "faible").astype(np.uint8)
tmp["additifs_n_risque_moyen"] = (tmp["risque"] == "moyen").astype(np.uint8)
tmp["additifs_n_risque_eleve"] = (tmp["risque"] == "eleve").astype(np.uint8)

# On aggrège les données par code du produit
cols = [
    "code",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
]

tmp = tmp[cols].groupby("code").aggregate("sum").reset_index()

# On merge les 2 DataFrames
df_clean = df_clean.reset_index().merge(
    tmp,
    how="left",
    on="code"
).set_index("index")

# Nous allons remplacer les valeurs manquantes par 0
df_clean = df_clean.fillna({
    "additifs_n_risque_faible": 0,
    "additifs_n_risque_moyen": 0,
    "additifs_n_risque_eleve": 0,
})

In [ ]:
print_md("Voici des exemples des nouvelles variables :")

cols = [
    "additives_fr",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
]

df_clean.dropna(subset=["additives_fr"])[cols].head()

### Création des variables `pn_*`

Nous allons utiliser le principe du target encoding afin de créer des variables numériques à partir de la variable de type catégorie `product_name`. Notre variable cible sera la catégorie du produit `pnns_groups_1`.

Pour chaques mots du nom d'un produits, nous allons leurs attribuer un score correspondant à la fréqeunce d'apparition du mot pour une des modalités de la varaible cible. Enfin, nous allons faire la somme des scores de chacun des mots du nom du produit pour chaque modalité de la variable cible.

Cela va nous permettre de projecter le nom d'un produit dans un espace où les dimensions représentent les catégories des produits. On obtiendra alors des variables de type numérique.

In [ ]:
df_clean["product_name"]

In [ ]:
# TODO ANalysie the whole cell
import nltk
nltk.download('stopwords')

# On crée un jeu de donnés pour entrainer les encodeurs
tmp = df_clean.dropna(subset=["pnns_groups_1"])
# On encode la variable cible de type catégorie
dummies = pd.get_dummies(tmp["pnns_groups_1"], prefix="pn")
dummies.columns = dummies.columns.str.lower().str.replace(" ", "_")

# On crée un jeu d'entrainement pour les encodeurs
tmp = tmp["product_name"].str.lower().str.replace("[^a-zA-Z ]", "").str.strip().str.split()
tmp = pd.concat([tmp, dummies], axis=1)
tmp = tmp.explode("product_name")

# On crée la liste des stop-words dans les prinicpales langues
words_to_del = []
for l in ["english", "french", "dutch", "spanish", "italian"]:
    words_to_del += stopwords.words(l)

# On supprime les stop-words
tmp = tmp[~tmp["product_name"].isin(words_to_del)]

# On crée la liste des encodeurs (un par modalité de la variable cible)
map_encoders = []

# Nous utiliseront de l'additive smoothing afin d'éviter l'over-fitting.
# m représentent le nombre d'individus minimums qu'il faut pour compenser
# l'additive smoothing ajouté par la moyenne générale.
m = 0

# On entraine les encodeurs (un par modalité de la variable cible)
for c in dummies.columns:
    # Calcul de la moyenne générale
    mean = tmp[c].mean()

    # Calcul du nombre d'individus et de la moyenne pour chaque groupe
    agg = tmp.groupby("product_name")[c].agg(["count", "mean"])
    counts = agg["count"]
    means = agg["mean"]

    # Calcule de la moyenne lissée
    smooth = (counts * means + m * mean) / (counts + m)

    map_encoders.append(smooth)

map_encoders = pd.concat(map_encoders, axis=1)

tmp = df_clean["product_name"].str.lower().str.replace("[^a-zA-Z ]", "").str.strip().str.split(expand=True)

# On encode les mots et on additionne le résultat pour chaque modalité de "pnns_groups_1"
for i in range(map_encoders.shape[1]):
    tmp2 = tmp.copy()
    for c in tmp2.columns:
        tmp2[c] = tmp2[c].map(map_encoders[i])
    df_clean[dummies.columns[i]] = tmp2.sum(axis=1)

In [ ]:
print_md("Voici des exemples des nouvelles variables :")

cols = [
    "product_name",
    "pnns_groups_1",
    "pn_beverages",
    "pn_cereals_and_potatoes",
    "pn_composite_foods",
    "pn_fat_and_sauces",
    "pn_fish_meat_eggs",
    "pn_fruits_and_vegetables",
    "pn_milk_and_dairy_products",
    "pn_salty_snacks",
    "pn_sugary_snacks",
]

df_clean.dropna(subset=["pnns_groups_1"])[cols].head()

## Types des variables

Certaines variables représentent un horodatage. Nous allons donc les convertir en type DateTime.

In [ ]:
# Conversion des variables de type datetime
c_t = [c for c in df_clean.columns if c.endswith("_t")]
    
for c in c_t: 
    df_clean.loc[:, [c]] = pd.to_datetime(df_clean[c], unit='s', errors="coerce")

Voyons les types de nos variables :

In [ ]:
df_clean.info(verbose=True, null_counts=True)

In [ ]:
# Liste des variables de type numérique
num_cols = df_clean.select_dtypes(include=np.number).columns.tolist()

# Liste des variables de type catégorie
cat_cols = df_clean.select_dtypes(include="object").columns.tolist()

text = "Nombre de variables de type :\n"
text += f"- **Numérique** : {len(num_cols)}\n"
text += f"- **Catégorie** : {len(cat_cols)}\n"

print_md(text)

In [ ]:
text = "Voici la liste des variables de type numérique :\n"

for c in num_cols:
    text += f"- `{c}`\n"

print_md(text)

In [ ]:
text = "Voici la liste des variables de type catégorie :\n"

for c in cat_cols:
    text += f"- `{c}`\n"

print_md(text)

## Sauvegarde du jeu de données nettoyé

In [ ]:
# On met les variables dans l'ordre
cols = [
    "code",

    "creator",
    "last_modified_t",

    "product_name",
    "brands",
    "countries_fr",
    "pnns_groups_1",
    "pnns_groups_2",

    "nutriscore_grade",
    "nutriscore_score",

    "ingredients_text",
    "ingredients_n",
    
    "additives_fr",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    
    "allergens",
    "allergens_n",

    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    
    "pn_beverages",
    "pn_cereals_and_potatoes",
    "pn_composite_foods",
    "pn_fat_and_sauces",
    "pn_fish_meat_eggs",
    "pn_fruits_and_vegetables",
    "pn_milk_and_dairy_products",
    "pn_salty_snacks",
    "pn_sugary_snacks"
]

df_clean = df_clean[cols]

In [ ]:
print_md_df_shape(df_clean, df)

In [ ]:
cols = [
    "product_name",
    "nutriscore_grade",
    "ingredients_text",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
]

# On supprime les individus ayant des valeurs manquantes dans les variables ci-dessus
df_clean = df_clean.dropna(subset=cols)

fig, ax = plt.subplots(figsize=(16, 8))

msno.bar(df_clean, ax=ax)
ax.set_title(
    """
    Représentation des valeurs présentes par variable.
    """,
    fontsize=16)
fig.axes[0].set_ylabel("Fréquence de valeurs présentes", fontsize=16)
fig.axes[1].set_ylabel("Nombre de valeurs présentes", fontsize=16)

# plt.savefig('valeurs présentes après nettoyage.png', bbox_inches='tight')
plt.show()

`brands`, `countries_fr`, `pnns_groups_1` et `pnns_groups_2` contiennent encore des valeurs manquantes.

`additives_fr` et `allergens` sont des listes et leurs valeurs manquantes représentent l'abscence d'additifs ou l'absence d'allergènes.

In [ ]:
# On sauvegarde le jeu de données nettoyé
df_clean.to_csv('df_clean.csv')

# Analyse univariée

## Chargement du jeu de données nettoyé

In [ ]:
df_clean = pd.read_csv("df_clean.csv",
    parse_dates=["last_modified_t"],
    low_memory=False
)

print_md_df_shape(df_clean)

## Analyse des variables de type numérique

### Distribution des variables

Commençons par observer les distributions des variables de type numérique :

In [ ]:
# On calcule les indicateurs statistiques de base
df_stats = df_clean.describe().T

# On réorganise et modifie le nom des colonnes
df_stats = df_stats[[
    "mean",
    "std",
    "min",
    "max",
    "25%",
    "50%",
    "75%"
]]
df_stats.columns = [
    "moyenne",
    "écart type",
    "min",
    "max",
    "Q1",
    "Q2",
    "Q3"
]

# On ajoute le nombre de valeurs manquantes
na_nb = []
for c in df_stats.index:
    na_nb.append(df_clean[c].isna().sum())

df_stats["valeurs manquantes"] = na_nb

# On ajoute le nombre de valeurs nulles
zero_nb = []
for c in df_stats.index:
    zero_nb.append((df_clean[c] == 0.).sum())

df_stats["valeurs nulles"] = zero_nb

# On ajoute une mesure de l'asymétrie de la distribution
df_stats["skewness"] = df_clean.skew()

In [ ]:
df_stats

In [ ]:
cols = list(df_stats.index)
cols_nb = len(cols)
c_nb = 3
r_nb = np.ceil(cols_nb / c_nb).astype(int)

sz = 16 / c_nb
fig = plt.figure(figsize=(sz *  c_nb, sz * r_nb))

for i, c in enumerate(cols):
    ax = plt.subplot(r_nb, c_nb, i + 1)
    ax.set_title(f"Distribution de {c}", fontsize=14)
    
    # Si la courbe est très étalée, on utilise une échelle log
    if df_stats.loc[c, "skewness"] > 2:
        ax.set_yscale("log")
        ax.set_ylabel("Occurrence (log)", fontsize=14)
    else:
        ax.set_ylabel("Occurrence", fontsize=14)

    df_clean[c].hist(bins=20, ax=ax)
    mean_line = ax.axvline(x=df_clean[c].mean(), linewidth=3, color='g', label="moyenne", alpha=0.7)
    med_line = ax.axvline(x=df_clean[c].median(), linewidth=3, color='y', label="médiane", alpha=0.7)
    
    plt.legend(handles=[mean_line, med_line], loc='upper right')

plt.tight_layout()
plt.show()

On remarque que seule la variable `nutriscore_score` semble suivre une loi normale multimodale. Ceci semble cohérent car cette variable est correlée à `nutriscore_grade` (voir la suite de l'analyse...) qui est une variable de type catégorie possédant 5 modalités.

### Comparaison des indicateurs statistiques

- `moyenne` : moyenne arithmétique
- `écart type` : écart-type empirique
- `min` : valeur minimale
- `max` : valeur maximale
- `Q1` : 1er quartile (25% des valeurs se trouvent en dessous)
- `Q2` : 2ème quartile (50% des valeurs se trouvent en dessous)
- `Q3` : 3ème quartile (75% des valeurs se trouvent en dessous)
- `valeurs manquantes` : nombre de valeurs manquantes
- `valeurs nulles` : nombre de valeurs à 0
- `skewness` : skewness empirique indiquant l'asymétrie de la distribution

In [ ]:
# On adapte le nombre de décimals affichées en fonction de la valeur de l'indicateur statistique
display(df_stats.style.format(lambda x: f"{x:.4f}" if abs(x) < 0.01 else f"{x:.2f}"))

Les produits ont en moyenne 14 ingrédients. Cela peut être compréhensible pour des plats cuisinés mais peut sembler beaucoup pour par exemple une mousse au chocolat.

On constate que certains produits ont jusqu'à 9 additifs considérés comme risqués pour la santé.

On constate aussi sur le tableau ci-dessus que les produits sont en moyenne principalement composés de glucides et de matières grasses.

Enfin, on remarque en observant le skewness que la distribution des valeurs est très élalée à droite sauf pour le `nutriscore_score`. Cela semble logique puisque la plupart des aliments sont composés d'une somme de nutriments. L'étalement à droite représente les produits spécifiques, comme par exemples les huiles, qui seront composés en majorité d'un seul nutriment.

### Répartition du nombre d'ingrédients par catégorie de produit

In [ ]:
num_categ_box_chart(
    df_clean,
    "ingredients_n",
    "pnns_groups_1",
    title="Répartition des valeurs de ingredients_n en fonction de pnns_groups_1",
    xlabel="Catégorie de produit",
    ylabel="Nombre d'ingrédients",
    showfliers=False,
    rotation=30
)

Sans surprise, on constate que les aliments composites (plats cuisinés etc...) sont ceux qui contiennent le plus d'ingrédients et les fuits et les légumes sont ceux qui en contiennet le moins.

### Produits ayant un nombre élevé d'additifs à risque

Voyons quelles sont les produits ayant jusqu'à 9 additifs considérés comme risqué pour la santé :

In [ ]:
# to debug the issue and nutriscore_grade column no data
cols = [
    "code",
    
    "creator",
    "last_modified_t",
    "product_name",
    "brands",
    "pnns_groups_1",

    "nutriscore_grade",

    "additives_fr",
    "additifs_n_risque_eleve",
]

df_clean[df_clean["additifs_n_risque_eleve"] >= 8][cols]

Ces produits sont des snacks sucrés vendus aux US. Il est intéressant de constater qu'ils ont été mis à jour en 2020 et sont donc certainement encore vendu en magasin.

### Comparaison des distribution des nutriments

In [ ]:
cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

fig, ax = plt.subplots(figsize=(16, 10))

sns.boxplot(
    data=df_clean[cols],
    showfliers=False,
    showmeans=True,
    ax=ax
)

ax.set_title("Comparaison des distributions des nutriments")
ax.set_ylabel("Quantité en g")

# plt.savefig('comparaison des distributions des nutriments.png', bbox_inches='tight')
plt.show()

On peut voir visuellement ce que l'on avait constaté dans le tableau des indicateurs statistiques (il y a en moyenne plus de glucides et de matières grasses dans nos aliments).

Il est aussi intéressant de constater les valeurs des quartiles Q3 indiquant que 75% des aliments sont en dessous de ces seuil :

In [ ]:
text = ""

for i, v in df_clean[cols].quantile(0.75).items():
    text += f"- `{i}` : {v:0.2f}g\n"
    
print_md(text)

## Analyse des variables de type catégorie

### Comparaison des indicateurs statistiques

- `nombre de valeurs` : nombre de valeurs présentes
- `nombre de modalités` : nombre de modalités de la variable
- `mode` : modalité qui apparait le plus souvent
- `effectif du mode` : nombre de fois où le mode apparait
- `fréquence du mode` : fréquence d'apparition du mode
- `valeurs manquantes` : nombre de valeurs manquantes

In [ ]:
df_clean.describe(include=['object']).T

In [ ]:
df_stats = df_clean[df_clean.columns.difference(["code"])].describe(include=['object']).T

df_stats.columns = [
    "nombre de valeurs",
    "nombre de modalités",
    "mode",
    "effectif du mode",
]

df_stats["fréquence du mode"] = df_stats["effectif du mode"] / df_stats["nombre de valeurs"]

# On ajoute le nombre de valeurs manquantes
inconnu_nb = []
for c in df_stats.index:
    inconnu_nb.append(df_clean[c].isna().sum())

df_stats["valeurs manquantes"] = inconnu_nb

display(df_stats)

Excepté `nutriscore_grade`, `pnns_groups_1` et `pnns_groups_2`, les autres variables ont toutes un grand nombre de modalités. Ceci est du au fait que ce sont des variables de type texte. Elles ne respectent pas de format spécifique et sont donc très irrégulières. Par exemple, pour représenter la valeur `Royaume-Uni`, on pourrait avoir dans nos données :
- Royaume-Uni
- royaume uni,france
- UK
- etc...

On rappelle aussi que certaines variables, comme `ingredients_text`, sont au format csv et représentent des listes de textes. Cela explique aussi pourquoi il y a de si grands nombres de modalités.

### Distribution de la variable `creator`

In [ ]:
# Doubt: cannot solve the issue
categ_plot_pie_chart(df_clean, "creator", 4)
# categ_plot_bar_chart(df_clean, "creator", 8)

In [ ]:
categ_plot_bar_chart(df_clean, "creator", 8)

57% des produits proviendraient du département de l'agriculture des US.

On a ensuite 16% des produits qui ont été rentré par `kiliweb` sur qui nous n'avont pas beaucoup d'information.

On retrouve ensuite des contributeurs anonymes avec 8% des produits.

### Distribution de la variable `product_name`

In [ ]:
get_cat_var_emp_dist_df(df_clean, "product_name", k=15)

Les noms qui apparaissent le plus souvent sont `Ice cream` et `Potato chips`. Les noms sont en anglais, il s'agit donc probablement de produits vendus aux US. Il est cependant vrai que, même en France, le rayons des chips comporte de nombreuses variétés et marques différentes.

La fréquence des effectifs reste très faible. En effet, le nom de chaque produit peut varier en fonction des ses particularités, de son pays de vente etc... Il faudrait donc nettoyer ces valeurs et les uniformiser afin de pouvoir les exploiter.

### Distribution de la variable `countries_fr`

In [ ]:
categ_plot_pie_chart(df_clean, "countries_fr", 2)

In [ ]:
categ_plot_bar_chart(df_clean, "countries_fr", 8)

59% des produits de notre jeu de données nettoyé sont vendus aux US et 23% en France.

Dans le jeu de données inital (avant nettoyage) ces chiffres étaient presques inversés. Il y a donc beaucoup de produits vendus en France qui ont des valeurs manquantes/aberrantes et qui ont été supprimés durant le nettoyage des données.

### Distribution de la variable `brands`

In [ ]:
tmp = df_clean[["brands"]].fillna("Unknown")

get_cat_var_emp_dist_df(tmp, "brands", k=10)

Les 5 marques les plus représentées sont française. Cependant, la variable `brands` contient 28% de valeurs manquante. Voyons dans quel pays sont vendus les produits n'ayant pas leur marque :

In [ ]:
pd.DataFrame(df_clean[df_clean["brands"].isna()]["countries_fr"].value_counts()[:5]).rename(
    columns={"countries_fr": "Nombre de produits sans marque"}
)

On constate que la grande majorité des produits n'ayant pas de valeur dans la variable `brands` sont vendus aux US.

### Distribution de la variable `pnns_groups_1`

In [ ]:
tmp = df_clean[["pnns_groups_1"]].fillna("Unknown")

categ_plot_pie_chart(tmp, "pnns_groups_1", 10)

In [ ]:
categ_plot_bar_chart(tmp, "pnns_groups_1", 10)

17% des produits sont des snacks sucrés.

Il est à noter que la catégorie la moins représentée est composée de 11000 produits. Il s'agit d'un nombre important de produits et on va donc considérer que chaque catégorie est représentative de sa population.

### Distribution de la variable `pnns_groups_2`

In [ ]:
tmp = df_clean[["pnns_groups_2"]].fillna("Unknown")

categ_plot_pie_chart(tmp, "pnns_groups_2", 15)

In [ ]:

categ_plot_bar_chart(tmp, "pnns_groups_2", 40, log=True)

Comme précédemment, on retrouve en tête du classement les biscuits et les gateaux avec 9% des produits et les sucreries avec 7% des produits.

On remaque cependant que le nombre de produits par catégorie décroit de façon exponentielle. Les dernières catégories contiennent moins de 1000 individus contre 33000 pour la première catégorie.

Nous allons privilégier `pnns_groups_1` pour le reste de cette analyse car elle a une répartition plus uniforme des individus par catégorie de produit.

### Distribution de la variable `nutriscore_grade`

In [ ]:
categ_plot_pie_chart(df_clean, "nutriscore_grade", 5)

In [ ]:
categ_plot_bar_chart(df_clean, "nutriscore_grade", 5)

Cette variable représente le Nutri-Score (code couleur avec les lettre A, B, C, D ou E) visible sur la plupart des embalages des produits. Le grade **d** semble le plus représenté avec 29% des produits.

Le grade le moins représenté est le grade **b** mais il est quand même attribué à environ 54000 produits.

### Distribution de la variable `ingredients_text`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des ingrédients.
tmp = pd.read_csv("df_clean_ingredients.csv", low_memory=False)
tmp.head(2)

get_cat_var_emp_dist_df(tmp, "ingredient", k=8)

Les ingrédients les plus utilisés sont le sel, le sucre et l'eau. On remarque que les 6 modalités les plus représentés sont des doublons : les 3 premières sont en anglais et les 3 suivantes sont en français. Ceci illustre bien le fait qu'il y a un gros travail à faire sur les valeurs des ingrédients afin de pouvoir exploiter cette variable. Nous ne l'utiliseront pas pour la suite de cette analyse.

### Distribution de la variable `additives_fr`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des additifs.
tmp = pd.read_csv("df_clean_additifs.csv", low_memory=False)
tmp.head(2)

categ_plot_pie_chart(tmp, "additif", 10)

In [ ]:
categ_plot_bar_chart(tmp, "additif", 20, log=True)

L'additif le plus utilisé est l'acide citrique qui est présent dans 8% des produits ayant des additifs. Ceci n'est pas étonnant puisqu'il s'agit d'un assaisonnement très fréquemment utilisé en cuisine (par exemple via l'utilisation du ju de citron).

In [ ]:
pct = df_clean[df_clean["additives_n"] > 0].shape[0] * 100 / df_clean.shape[0]

print_md(f"""Il est a noté que les chiffres ci-dessus concernent les produits ayant des additifs, 
soit {int(pct)}% des produits du jeu de données.""")

### Distribution de la variable `allergens`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des allergènes.
tmp = pd.read_csv("df_clean_allergens.csv", low_memory=False)
tmp.head(2)

get_cat_var_emp_dist_df(tmp, "allergen", k=6)

29% des allergène sont de type produit laitier. En retrouve ensuite le gluten avec 24% et le soja avec 10% des allergènes.

In [ ]:
pct = df_clean[df_clean["allergens_n"] > 0].shape[0] * 100 / df_clean.shape[0]

print_md(f"""Il est a noté que les chiffres ci-dessus concernent les produits ayant des allergènes, 
soit {int(pct)}% des produits du jeu de données.""")

## Comparaison entre la France et les US

Les 2 pays les plus représentés sont la France et les US. Continuons notre analyse en observant les différences des distributions entre ces 2 pays.

### Evolution du nombre de produits ajoutés dans le jeu de données

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))

for p in ["États-Unis", "France"]:
    # On récupère les produits vendus dans le pays p
    tmp = df_clean[df_clean["countries_fr"] == p][["last_modified_t"]]. \
        sort_values("last_modified_t").copy()
    
    # On ajoute le nombre de produits présents à chaque date
    tmp[p] = 1
    tmp[p] = tmp[p].cumsum()
    
    tmp = tmp.set_index("last_modified_t")
    
    tmp.plot(ax=ax)

ax.set_title("Evolutions du nombre de produits ajoutés dans le jeu de données par pays de vente")
ax.set_xlabel("Date")
ax.set_ylabel("Nombre total de produits")
ax.set_yscale("log")
    
# plt.savefig('Evolutions du nombre de produits ajoutés dans le jeu de données par pays de vente.png', bbox_inches='tight')
plt.show()

Les premiers produits français ont été ajouté en 2012. Depuis, il y a eu des ajout de produits réguliers qui suivent une courbre exponentielle (l'échelle des ordonnées est en log).

Les premiers produits des US ont été ajoutés en 2015. On observe des pics d'ajouts de produits. Il s'agit certainement d'upload de données venant d'une autre base de données.

Les produits et leurs compositions changent régulièrement. Pour cette partie de l'analyse, nous allons donc comparer les produits ajoutés à partir de 2015. Nous allons aussi prendre au hazard 2000 produits dans chaque catégorie de produits de la variable `pnns_groups_1`.

In [ ]:
# On filtre par date de dernière modification du produit et par pays
df_sample = df_clean[df_clean["last_modified_t"].dt.year >= 2015]
df_sample = df_sample[df_sample["countries_fr"].isin(["France", "États-Unis"])]

# On va prendre au hazard 2000 produits dans chaque catégorie de produits
df_sample = df_sample.groupby(
    ["countries_fr", "pnns_groups_1"],
    group_keys=False
).apply(lambda x: x.sample(min(len(x), 2000)))

outs = []

# On affiche les distribution empirique des catégories de produits
for p in ["France", "États-Unis"]:
    outs.append(widgets.Output())

    with outs[-1]:
        tmp = get_cat_var_emp_dist_df(
            df_sample[df_sample["countries_fr"] == p],
            "pnns_groups_1"
        )

        print_md(f"<center>Distribution empirique des </br>modalités de pnns_groups_1 dans le pays {p}</center>")
        display(tmp)

hbox = widgets.HBox(outs)
hbox.layout.justify_content="space-around"

display(hbox)

### Répartition des nutriments

In [ ]:
def num_box_chart(
        df,
        num_cols,
        cat_col,
        cat_include,
        title="",
        xlabel="",
        ylabel="",
        save=False
    ):
    # On filtre les données numériques et la catégorie qui nous intéresse
    tmp = df[df[cat_col].isin(cat_include)][num_cols + [cat_col]]

    # On transforme les colonnes numériques en catégorie
    tmp = tmp.melt(
        id_vars=cat_col,
        var_name=xlabel,
        value_name=ylabel
    )

    fig, ax = plt.subplots(figsize=(16, 10))
    
    meanprops = {
        'marker':'o',
        'markeredgecolor':'black',
        'markerfacecolor':'firebrick'
    }

    sns.boxplot(
        data=tmp,
        x=xlabel,
        y=ylabel,
        hue=cat_col,
        showfliers=False,
        showmeans=True,
        meanprops=meanprops,
        ax=ax
    )
    
    ax.set_title(title)
    ax.set_xlabel("")

    if save:
        plt.savefig(f"{title}.png", bbox_inches='tight')
    
    plt.show()

num_cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g"
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition des nutriments par pays",
    xlabel="type de nutriment",
    ylabel="nutriments en g",
#     save=True
)

On constate que les répartition des proportions entre les différents types de nutriments sont similaires entre les 2 pays.

On notera qu'il y a une plus grande proportion de produits aux US qui contiennent plus de glucides qu'en France.

### Répartition du nombre d'ingrédients

In [ ]:
num_cols = [
    "ingredients_n",
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition du nombre d'ingrédients par pays",
    xlabel="ingrédients",
    ylabel="nombre d'ingrédients"
)

On constate qu'il y a un peu plus d'ingrédients dans les produits vendus aux US qu'en France :
- Aux **US**, 75% des produits ont moins de **19 ingrédients**.
- En **France** 75% des produits ont moins de **14 ingrédients**.

### Répartition du nombre d'additifs et d'allergènes

In [ ]:
num_cols = [
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition du nombre d'additifs et d'allergènes par pays",
    xlabel="additifs et allergènes",
    ylabel="nombre d'additifs ou d'allergènes",
#     save=True
)

25% des produits vendus en France ont au moins 1 allergène.

# Analyse bivariée

## Correlation entre `nutriscore_score` et `nutriscore_grade`

In [ ]:
num_categ_box_chart(
    df_clean,
    "nutriscore_score",
    "nutriscore_grade",
    title="Répartition des valeurs de nutriscore_score en fonction de nutriscore_grade",
    xlabel="Nutri-Score",
    ylabel="Score"
)

Il semble effectivement qu'il y ait une relation linéaire entre `nutriscore_score` et `nutriscore_grade`.

Nous allons effectuer une analyse de la variance (ANOVA) afin de quantifier cette corrélation.

In [ ]:
# Doubt : undertstand the cell
# Rapport de corrélation
def eta_squared(df, x, y):
    tmp = df.dropna(subset=[x, y])
    x, y = tmp[x], tmp[y]
    
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    # Variation totale
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    
    # Variation interclasse
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    
    return SCE/SCT
    
c = eta_squared(df_clean, "nutriscore_grade", "nutriscore_score")
print_md(f"Le rapport de correlation entre `nutriscore_score` et `nutriscore_grade` est de : **{c:0.2f}**")

Il y a clairement une forte correlation entre ces 2 variables.

On remarque cepdendant la présence d'outliers, notamment pour les nutriscore de grade **a** et **b**. Voyons quelques exemples de ces produits :

In [ ]:
df_clean[(df_clean["nutriscore_grade"] == "a") & (df_clean["nutriscore_score"] > 4)]

On remarque que la plupart des outliers du grade **a** sont des eaux minérales ([en savoir plus sur le calcul du Nutri-Score](https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score)).

## Analyse des correlations

### Correlations entre les variables de type numérique et les variables de type catégorie

In [ ]:
rows = [
    "nutriscore_grade",
    "pnns_groups_1",
    "pnns_groups_2",
]

cols = [
    "nutriscore_score",
    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
]

data = {}

for r in rows:
    data[r] = []
    for c in cols:
        es = eta_squared(df_clean, r, c)
        data[r].append(es)

tmp = pd.DataFrame.from_dict(data, orient='index', columns=cols)

fig, ax = plt.subplots(figsize=(6, 6))

sns.heatmap(tmp.T, annot=True, ax=ax)

ax.set_title("Matrice des corrélations entre les variables de type numérique et 3 variables de type catégorie")

plt.show()

On observe des correlations modérées entre `nutriscore_score`, `energy_100g`, `carbohydrates_100g` et les catégorie de produits. On remarque que plus on a de modalités dans les catégories (`pnns_groups_2`), plus ces correlations sont fortes.

Cela semble cohérent car certaines catégories de produits, comme les snacks sucrés, vont avoir une énergie aux 100g très élevée et un nutriscore très mauvais. Alors que des catégories comme les fruits et les légumes vont certainement regrouper des produits ayant une énergie aux 100g faible et un très bon nutriscore.

In [ ]:
num_categ_box_chart(
    df_clean,
    "energy_100g",
    "pnns_groups_1",
    title="Répartition des valeurs de energy_100g en fonction de pnns_groups_1",
    xlabel="Catégorie des produits",
    ylabel="Energie en kJ",
    showfliers=False,
    rotation=30
)

On constate effectivement que la majorité des fruits et légumes ont une énergie aux 100g assez basse contrairement aux snacks sucrés.

In [ ]:
num_categ_box_chart(
    df_clean,
    "nutriscore_score",
    "pnns_groups_1",
    title="Répartition des valeurs de nutriscore_score en fonction de pnns_groups_1",
    xlabel="Catégorie des produits",
    ylabel="Score de nutrition",
    showfliers=False,
    rotation=30
)

On constate de même que le nutriscore de la majorité des fruits et légumes est bas contrairement à celui des snacks sucrés.

### Correlations entre les variables de type numérique

In [ ]:
cols = [
    "nutriscore_score",
    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
]

corr = df_clean[cols].corr()

fig, ax = plt.subplots(figsize=(14, 12))

sns.heatmap(corr, annot=True, ax=ax)
ax.set_title("Matrice des correlations des variables de type numérique")

plt.show()

Les coefficients de correlation ci-dessus nous donne les informations suivantes :
- Correlation positive élevée entre le nombre d'ingrédients et le nombre d'additifs
- Correlation positive élevée entre le nombre d'additifs et le nombre d'additifs ayant un risque élevé pour la santé
- Corrélations positives modérées entre le score de nutrition et :
    - L'énergie aux 100g
    - Les matières grasses
    - Les matières grasses saturées
    - Les sucres

### Correlations entre les variables de type catégorie

In [ ]:
def corr_cat_cat_heatmap(df, c1, c2, title):
    # On supprimes les individus ayant des valeurs manquantes
    tmp = df.dropna(subset=[c1, c2])

    # On crée le tableau de contingence
    cont = tmp[[c1, c2]].pivot_table(
        index=c1,
        columns=c2,
        aggfunc=len,
        margins=True,
        margins_name="Total",
    )
    cont

    tx = cont.loc[:,["Total"]]
    ty = cont.loc[["Total"],:]
    n = len(tmp)
    indep = tx.dot(ty) / n

    c = cont.fillna(0) # On remplace les valeurs nulles par 0
    measure = (c - indep) ** 2 / indep
    xi_n = measure.sum().sum()
    table = measure / xi_n
    
    h = 0.6 * len(cont)
    fig, ax = plt.subplots(figsize=(14, h))
    
    sns.heatmap(table.iloc[:-1,:-1], annot=c.iloc[:-1,:-1], ax=ax)
#     sns.heatmap(table.iloc[:-1,:-1], annot=table.iloc[:-1,:-1], ax=ax)
    
    ax.set_title(title)
    
    plt.show()

corr_cat_cat_heatmap(
    df_clean,
    "pnns_groups_1",
    "nutriscore_grade",
    "Matrice des correlations entre les modalités des variables pnns_groups_1 et nutriscore_grade"
)

Comme nous l'avons supposé, on observe effectivement des correlations entre les catégories des produits et le Nutri-Score. On remarquera notamment :
- Corrélation entre la catégorie **céréales et pommes de terre** et le Nutri-Score **a**.
- Corrélation entre la catégorie **Fruits et légumes** et le Nutri-Score **a**.
- Corrélation entre la catégorie **Snacks sucrés** et le Nutri-Score **e**.

Il est aussi intéressant de constater qu'il y a des produits du type **Fruits et légumes** qui ont un grade **d** ! Observons quelques-un des ces produits :

In [ ]:
df_clean[(df_clean["pnns_groups_1"] == "Fruits and vegetables") & (df_clean["nutriscore_grade"] == "d")]

Il sagit de produits riches (comme les oléagineux) mais constitués principalement de fruits et de légumes.

# Analyse multivariée

## Analyse en composantes principales (ACP) 

In [ ]:
cols = [
    "nutriscore_score",

    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",

    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    
    "pn_beverages",
    "pn_cereals_and_potatoes",
    "pn_composite_foods",
    "pn_fat_and_sauces",
    "pn_fish_meat_eggs",
    "pn_fruits_and_vegetables",
    "pn_milk_and_dairy_products",
    "pn_salty_snacks",
    "pn_sugary_snacks",
]

# On ne conserve que les individus ayant toutes les valeurs des variables ci-dessus
acp_df = df_clean.dropna(subset=cols)

In [ ]:
X = acp_df[cols].values
names = acp_df["code"].to_numpy()
features = cols

# On transforme nos variables en variables centrées réduites
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

In [ ]:
acp_df[cols].values

In [ ]:
X_scaled

In [ ]:
X_scaled

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# On réalise une PCA
pca = decomposition.PCA(n_components=len(features), random_state=0)
pca.fit(X_scaled)

# On récupère le ratio de l'inertie de chaque axe d'inertie
evrs = pca.explained_variance_ratio_

# On crée et on affiche le diagramme des éboulis de valeurs propres
fig, ax = plt.subplots(figsize=(16,10))

ax.set_title("Diagramme des éboulis de valeurs propres avec somme cumulée")
ax.set_xlabel("Rang de l'axe d'inertie")
ax.set_ylabel("Taux d'inertie")

ax.bar(np.arange(len(evrs)) + 1, evrs)
l1 = ax.plot(np.arange(len(evrs)) + 1, evrs.cumsum(), c="red", marker='o', label="Inertie cumulée")[0]

# Ligne représentant le critère de Kaiser
k = 1 / len(features)
l2 = ax.axhline(k, color="grey", ls="--", alpha=0.7, label="Critère de Kaiser")

# On ajoute le taux d'inertie
for i, evr in enumerate(evrs):
    if evr > k:
        ax.text(i + 0.65, evr + .02, f"{evr:0.2f}")
        
plt.legend(handles=[l1, l2], loc='upper left')

plt.show()

Les 4 premiers axes d'inertie permettent d'expliquer 42% de la variance totale. Les autres axes ont un taux d'intertie trop faible et qui passe rapidement en dessous du critère de Kaiser.

Nous allons continuer cette ACP avec les 4 premiers axes d'inertie F1, F2, F3 et F4. Commençons par observer nos variables projectées sur le 1er plan factoriel (F2, F1).

In [ ]:
def pca_plot_var(pcs, x_idx, y_idx, circle=False):
    # On affiche le cercle de corrélation de chaque plan factoriel
    fig, ax = plt.subplots(figsize=(14, 14))

    for j, (x, y) in enumerate(zip(pcs[x_idx, :], pcs[y_idx, :])):
        ax.plot([0, x], [0, y], color="grey", alpha=0.7)
        plt.text(x, y, cols[j], fontsize=14)

    if circle:
        ax.add_patch(plt.Circle((0, 0), 1, color="grey", alpha=0.7, fill=False))
    
    ax.axhline(0, color="grey", ls="--", alpha=0.5)
    ax.axvline(0, color="grey", ls="--", alpha=0.5)

    ax.set_title(f"Cercle des corrélations sur le plan factoriel (F{x_idx + 1}, F{y_idx + 1})")
    ax.set_xlabel(f"F{x_idx + 1} ({evrs[x_idx] * 100: 0.1f}%)")
    ax.set_ylabel(f"F{y_idx + 1} ({evrs[y_idx] * 100: 0.1f}%)")
    
    ax.set_aspect("equal")

    plt.show()

pcs = pca.components_
X_projected = pca.transform(X_scaled)
    
pca_plot_var(pcs, 1, 0)

On constate que les variables qui contribuent le plus à F1 sont `energy_100g`, `nutriscore_score`. On retrouve ensuite `pn_sugary_snacks`, `saturated-fat_100g` et `fat_100g`. On ramqaure que `pn_fruits_and_vegetables` apporte une faible contribution négative. F1 pourrait donc représenter la notion de richesse énergétique d'un produit en termes de matières grasses et de sucres.

F2 augmente avec `additives_n` et diminue avec `fat_100g`, `saturated-fat_100g` et `proteins_100g`. F2 pourrait ainsi représenter les additifs qui feraient diminuer la quantité de matières grasses. F2 pourrait aussi simplement représenter les produits qui se différencie ou non des huiles qui sont des produits riches et ayant peu d'ingrédients.

Voyons maintenant la projection des variables initiales sur (F4, F3)

In [ ]:
pca_plot_var(pcs, 3, 2)

F3 augmente avec `ingredients_n` et `pn_composite_foods` et diminue avec `sugars_100g`. F3 semble représenter les produits salés ayant de nombreux ingrédients.

F4 augmente un peu avec `additifs_n_risque_faible` et `additifs_n_risque_moyen` et diminue avec `pn_cereals_and_potatoes` et `carbohydrates_100g`. F4 pourrait indiquer la proportion d'additifs qui permettrait de remplacer les glucides.

In [ ]:
tmp = acp_df[cols].copy()
for i in range(4):
    tmp[f"F{i + 1}"] = X_projected[:, i]

corrMatrix = tmp.corr()

fig, ax = plt.subplots(figsize=(14, 14))

sns.heatmap(corrMatrix.iloc[:-4, -4:], annot=True, ax=ax)
ax.set_title("Matrice des corrélation avec les axes d'inertie de l'ACP")

plt.show()

On peut constater sur la matrice des corrélation ci-dessus, ce que l'on avait observé sur les cercles des corrélations.

Nos nouvelles variables F1, F2, F3 et F4 sont des combinaisons linéaires des variables initiales. Les interprétations ci-dessus ne sont que des hypothèses faites à partir de l'observation des poids des variables initiales dans le calcul des variables F1, F2, F3 et F4. On pourrait vérifier ces hypothèses en projetant des individus dans nos plans factoriels. Voyons donc quelques produits projetés sur les 3 premiers axes d'inerties F1, F2 et F3 :

In [ ]:
X_projected = pca.transform(X_scaled)

# On crée un DataFrame avec nos individus et en leurs ajoutant les variables F1, F2 et F3
tmp = pd.DataFrame(X_projected[:, :3], columns=[f"F{i + 1}" for i in range(3)])
tmp["nutriscore_grade"] = acp_df["nutriscore_grade"].values
tmp["pnns_groups_1"] = acp_df["pnns_groups_1"].values
tmp = tmp.dropna()

# On va prendre au hazard 100 produits pour chaque Nutri-Score.
# Il serait trop lourd d'afficher tous les individus sur un digramme interractif en 3D.
tmp = tmp.groupby(
    ["nutriscore_grade", "pnns_groups_1"],
    group_keys=False
).apply(lambda x: x.sample(min(len(x), 100)))

# On affiche les individus en 3D
fig = px.scatter_3d(
    tmp,
    x="F1",
    y="F2",
    z="F3",
    color="pnns_groups_1",
    title=f"Représentation d'un échantillon des individus projetés sur (F1, F2, F3)"
)
fig.show()

En faisant tourner la figure ci-dessus, on remarque que certains groupes de produits sont facilement discernables comme par exemple :
- Les poissons, viandes et oeufs.
- Les boissons.
- Les produits composés.
- Les snacks sucrés.

Si on prend par exemple les individus extrêmes sur F1, on obtient :
- Un fruit ou légume avec une valeur négative sur F1.
- Un snack sucré avec une valeur positive sur F1.

Cela semble cohérent au vu de l'interprétation de F1 qui indiquerait la richesse énergétique d'un produit en termes de sucres et de matières grasses.

Dans le cadre de ce projet, les nouvelles variables crées grâce aux axes d'intertie pourraient servir de filtre lorsque les agents de Santé publique France voudront explorer le jeu de données.

## Recherche de groupes de produits via l'algorithme k-means

### Recherche du nombre optimal de clusters

Nous allons utiliser la méthode du coude qui permet de trouver le nombre de cluster à partir duquel l'inertie intraclasse ne varie plus beaucoup.

In [ ]:
# Colab crashes with this code
# c_max = 30
# res = []

# for i in range(c_max):
#     km = KMeans(n_clusters=i+1)
#     km.fit(X_scaled)
#     res.append(km.inertia_)

In [ ]:
# fig, ax = plt.subplots(figsize=(14, 10))

# ax.plot(res)
# ax.set_title("Représentation de l'inertie intraclasse en fonction du nombre de clusters")
# ax.set_xlabel("Nombre de clusters")
# ax.set_ylabel("Inertie intraclasse")
# ax.set_xticks(np.arange(c_max))
# ax.set_xticklabels(np.arange(c_max) + 1)

# plt.show()

Au vu du graphique ci-dessus, nous allons fixer les nombre de clusters à **9**. Cette valeur se trouve dans le coude du graphique et correspond aussi au nombre de modalités de `pnns_groups_1`.

### Analyse des clusters

In [ ]:
# # On réentraine le modèle avec le nombre optimal de custers
# c_optimal = 9

# # On fixe le random_state afin que l'on puisse reproduire les résultats.
# # En effet, les centroïdes sont initialiséa aléatoirement.
# # Modifier cette valeur pour optenir des résultats différents.
# km = KMeans(c_optimal, random_state=0)
# km.fit(X_scaled)

# km_df = acp_df.copy()

# # On ajoute les axes d'inertie trouvés lors de l'ACP
# for i in range(4):
#     km_df[f"F{i + 1}"] = X_projected[:, i]

# # On attribue à chaque individu un cluster
# km_df["cluster"] = km.labels_.astype(str)

In [ ]:
# # On va prendre au hazard 200 produits pour chaque cluster.
# # Il serait trop lourd d'afficher tous les individus sur un digramme interractif en 3D.
# tmp = km_df.groupby(
#     ["cluster"],
#     group_keys=False
# ).apply(lambda x: x.sample(min(len(x), 200)))

# # On affiche les individus en 3D
# fig = px.scatter_3d(
#     tmp,
#     x="F1",
#     y="F2",
#     z="F3",
#     color="cluster",
#     title=f"Représentation d'un échantillon des individus projetés sur (F1, F2, F3)"
# )
# fig.show()

On remarque que les clusters semblent facilement identifiables même si les individus ne sont projetés que sur F1, F2 et F3.

Cela peut sembler cohérent puisque notre algorithme de clusturing a utilisé les mêmes variables que celles utilisées lors de l'ACP.

Observons les corrélations entre les clusters et les catégories des produits :

In [ ]:
# corr_cat_cat_heatmap(
#     km_df,
#     "cluster",
#     "pnns_groups_1",
#     "Matrice des correlations entre les modalités des variables pnns_groups_1 et cluster"
# )

6 clusters semblent avoir une corrélation avec une catégorie de produit en particulier :
- Cluster 0 : céréales et pommes de terre.
- Cluster 1 : boissons.
- Cluster 2 : snacks sucrés.
- Cluster 5 : poissons, viandes et oeufs.
- Cluster 7 : produits composés.
- Cluster 8 : snacks salés.

Observons pour chaques clusters ayant une corrélation avec une catégorie de produits, les mots qui reviennent le plus souvent dans les noms des produits :

In [ ]:
# def make_word_cloud(data, cluster, subplotax, title):
#     words = data[data["cluster"] == cluster]["product_name"].apply(lambda l: l.lower().split())
#     cluster_words = words.apply(pd.Series).stack().reset_index(drop=True)
#     frequencies = cluster_words.value_counts()
    
#     text = " ".join(w for w in cluster_words)

#     # Create and generate a word cloud image:
#     wordcloud = WordCloud(max_font_size=40, max_words=30,
#                           background_color="white", colormap="magma")
#     wordcloud.generate_from_frequencies(frequencies)
#     # Display the generated image:
#     subplotax.imshow(wordcloud, interpolation='bilinear')
#     subplotax.axis("off")
#     subplotax.set_title(title)
#     return subplotax

# tmp = km_df.dropna(subset=["product_name"])

# cluster_to_category = {
#     "0": "céréales et pommes de terre",
#     "1": "boissons",
#     "2": "snacks sucrés",
#     "5": "poissons, viandes et oeufs",
#     "7": "produits composés",
#     "8": "snacks salés",
# }

# col_nb = 2
# row_nb = np.ceil(len(cluster_to_category) / col_nb).astype(int)

# fig = plt.figure(figsize=(16, 5 * row_nb))

# for i, (cluster, category) in enumerate(cluster_to_category.items()):
#     ax = plt.subplot(row_nb, col_nb, i + 1)
    
#     make_word_cloud(tmp, cluster, ax, f"Cluster {cluster} corrélé aux {category}")

# plt.show()

On voit effectivement que la plupart des mots représentent bien la catégorie de produit avec laquelle le cluster est corrélé.

Nous avons utilisé toutes les variables disponibles dans notre jeu de données nettoyé pour la recherche de 9 groupes de produits. Cette méthode pourrait permettre aux agents de Santé publique France de rechercher automatiquement d'autres groupes de produits selon des critères spécifiques. Par exemple, on pourrait ne prendre en compte qu'une poignée de variables et une catégorie spécifique de produits etc...